In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 20)

from time import sleep
import requests

import geopandas as gpd

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [13]:
for i in assembly_district:
    print(i)

23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [45]:
dataframe_list = []

assembly_district = range(23, 88)

for i in assembly_district:
    landing_endpoint = f'https://enr.boenyc.gov/CD26825AD{i}0.html'
    response = requests.get(landing_endpoint, headers=header)
    dfs = pd.read_html(response.content)
    dataframe = pd.read_html(response.content)[2].loc[2:].drop(columns=[2,4,6,8,10,12])
    dataframe['assembly_district'] = i
    dataframe.columns=['election_district', 'reported_votes', 'dem', 'gop', 'conservative', 'working_family', 'write_in', 'assembly_district']
    dataframe_list.append(dataframe)
    print(f'Finish AD {i}')
    sleep(1)


Finish AD 23
Finish AD 24
Finish AD 25
Finish AD 26
Finish AD 27
Finish AD 28
Finish AD 29
Finish AD 30
Finish AD 31
Finish AD 32
Finish AD 33
Finish AD 34
Finish AD 35
Finish AD 36
Finish AD 37
Finish AD 38
Finish AD 39
Finish AD 40
Finish AD 41
Finish AD 42
Finish AD 43
Finish AD 44
Finish AD 45
Finish AD 46
Finish AD 47
Finish AD 48
Finish AD 49
Finish AD 50
Finish AD 51
Finish AD 52
Finish AD 53
Finish AD 54
Finish AD 55
Finish AD 56
Finish AD 57
Finish AD 58
Finish AD 59
Finish AD 60
Finish AD 61
Finish AD 62
Finish AD 63
Finish AD 64
Finish AD 65
Finish AD 66
Finish AD 67
Finish AD 68
Finish AD 69
Finish AD 70
Finish AD 71
Finish AD 72
Finish AD 73
Finish AD 74
Finish AD 75
Finish AD 76
Finish AD 77
Finish AD 78
Finish AD 79
Finish AD 80
Finish AD 81
Finish AD 82
Finish AD 83
Finish AD 84
Finish AD 85
Finish AD 86
Finish AD 87


In [136]:
df = pd.concat(dataframe_list).reset_index(drop=True)

# Organize the order of the column
df = df[['assembly_district', 'election_district', 'reported_votes', 'dem', 'gop', 'conservative', 'working_family', 'write_in']]

df.head()

,assembly_district,election_district,reported_votes,dem,gop,conservative,working_family,write_in
0,23,ED 1,99.00%,257,826,141,12,12
1,23,ED 2,99.00%,240,792,115,19,11
2,23,ED 3,99.00%,84,285,33,6,3
3,23,ED 4,99.00%,337,771,55,15,13
4,23,ED 5,99.00%,290,831,56,20,10


In [137]:
# Change the data type from string to integer for certain columns
df.iloc[:, 3:] = df.iloc[:, 3:].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4484 entries, 0 to 4483
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assembly_district  4484 non-null   int64 
 1   election_district  4484 non-null   object
 2   reported_votes     4419 non-null   object
 3   dem                4484 non-null   object
 4   gop                4484 non-null   object
 5   conservative       4484 non-null   object
 6   working_family     4484 non-null   object
 7   write_in           4484 non-null   object
dtypes: int64(1), object(7)
memory usage: 280.4+ KB


In [138]:
# Calculate the total votes for both candidates
df['harris'] = df.dem + df.working_family
df['trump'] = df.gop + df.conservative

In [139]:
# Calcuate the vote share
df['harris_share'] = df.apply(lambda row: row['harris'] / (row['harris'] + row['trump'] + row['write_in']) if (row['harris'] + row['trump'] + row['write_in']) != 0 else 0, axis=1)
df['trump_share'] = df.apply(lambda row: row['trump'] / (row['harris'] + row['trump'] + row['write_in']) if (row['harris'] + row['trump'] + row['write_in']) != 0 else 0, axis=1)

In [140]:
df = df[['assembly_district', 'election_district', 
         'harris', 'harris_share',
         'trump', 'trump_share']]

In [141]:
# Save aseembly district data into a new dataframe
ad_df = df[df.election_district =='Total'].reset_index(drop=True)

# Another new dataframe for only election district data
ed_df = df[df.election_district !='Total'].reset_index(drop=True)

In [142]:
ed_df.assembly_district = ed_df.assembly_district.astype(str)

ed_df['election_district'] = ed_df['election_district'].str.replace(r'ED (\d+)', lambda x: x.group(1).zfill(3), regex=True)

ed_df['ed'] = ed_df.assembly_district + ed_df.election_district

ed_df.head()

,assembly_district,election_district,harris,harris_share,trump,trump_share,ed
0,23,001,269,0.215545,967,0.774840,23001
1,23,002,259,0.220051,907,0.770603,23002
2,23,003,90,0.218978,318,0.773723,23003
3,23,004,352,0.295550,826,0.693535,23004
4,23,005,310,0.256835,887,0.734880,23005


In [143]:
geo_file = 'NYS_Elections_Districts_and_Polling_Locations_-3344009424923235749.geojson'

gdf = gpd.read_file('/Users/keenansmacbookairm3/Downloads/'+ geo_file)

gdf.columns = gdf.columns.str.lower()

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13335 entries, 0 to 13334
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   objectid           13335 non-null  int32   
 1   county             13335 non-null  object  
 2   municipality       13335 non-null  object  
 3   election_district  13335 non-null  object  
 4   geometry           13333 non-null  geometry
dtypes: geometry(1), int32(1), object(3)
memory usage: 468.9+ KB


In [144]:
gdf = gdf[gdf.municipality.str.contains('New York City')]

gdf.shape

(4345, 5)

In [145]:
gdf.head(10)

,objectid,county,municipality,election_district,geometry
8955,9019,Queens,New York City,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.56148, -73.91989 40.56168, -73.91986 40.56148, -73.91982 40.56127, -73.91993 40.56126, -73.92004 40.56125, -73.92014 40.56123, -73.92013 40.561, -73.9201..."
8956,9020,Queens,New York City,23002,"POLYGON ((-73.91018 40.56493, -73.91039 40.56452, -73.91037 40.56441, -73.91055 40.56413, -73.9106 40.56403, -73.91065 40.56393, -73.91069 40.56384, -73.9107 40.56342, -73.91027 40.5624, -73.90958..."
8957,9021,Queens,New York City,23003,"POLYGON ((-73.86287 40.56684, -73.86268 40.56653, -73.86268 40.56652, -73.86269 40.56651, -73.86269 40.56651, -73.86271 40.56651, -73.86272 40.56651, -73.86277 40.56658, -73.86296 40.56682, -73.86..."
8958,9022,Queens,New York City,23004,"POLYGON ((-73.85274 40.57061, -73.85312 40.57051, -73.85332 40.57044, -73.85358 40.57035, -73.85381 40.57034, -73.85385 40.57034, -73.85397 40.57032, -73.8541 40.57026, -73.85423 40.57015, -73.854..."
8959,9023,Queens,New York City,23005,"POLYGON ((-73.84878 40.57206, -73.84887 40.57202, -73.84897 40.57199, -73.84902 40.57196, -73.84907 40.57195, -73.84914 40.57192, -73.84925 40.57186, -73.84936 40.57183, -73.84941 40.5718, -73.849..."
8960,9024,Queens,New York City,23006,"POLYGON ((-73.84527 40.57333, -73.84616 40.57304, -73.84623 40.57302, -73.847 40.57273, -73.84777 40.57241, -73.84878 40.57206, -73.84924 40.57298, -73.85003 40.57265, -73.85116 40.57461, -73.8511..."
8961,9025,Queens,New York City,23007,"POLYGON ((-73.84621 40.58177, -73.84623 40.58168, -73.84505 40.58147, -73.84506 40.58137, -73.84401 40.57952, -73.84305 40.57786, -73.84179 40.57568, -73.84175 40.57561, -73.84138 40.57473, -73.84..."
8962,9026,Queens,New York City,23008,"POLYGON ((-73.83632 40.57662, -73.83641 40.57658, -73.83704 40.57629, -73.83711 40.57626, -73.83766 40.57602, -73.83779 40.57593, -73.83801 40.57586, -73.83826 40.5758, -73.83843 40.57571, -73.838..."
8963,9027,Queens,New York City,23009,"POLYGON ((-73.82928 40.57887, -73.82957 40.57879, -73.82962 40.57878, -73.82968 40.57878, -73.82974 40.57878, -73.8299 40.57878, -73.83003 40.57878, -73.83017 40.57876, -73.83032 40.57871, -73.830..."
8964,9028,Queens,New York City,23010,"POLYGON ((-73.82134 40.58127, -73.82195 40.58118, -73.82214 40.58114, -73.82223 40.58112, -73.82229 40.5811, -73.82242 40.58105, -73.82265 40.58095, -73.8232 40.58076, -73.82321 40.58075, -73.8237..."


In [146]:
gdf[gdf.county =='Queens'].sort_values('election_district', ascending=True)

,objectid,county,municipality,election_district,geometry
8955,9019,Queens,New York City,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.56148, -73.91989 40.56168, -73.91986 40.56148, -73.91982 40.56127, -73.91993 40.56126, -73.92004 40.56125, -73.92014 40.56123, -73.92013 40.561, -73.9201..."
8956,9020,Queens,New York City,23002,"POLYGON ((-73.91018 40.56493, -73.91039 40.56452, -73.91037 40.56441, -73.91055 40.56413, -73.9106 40.56403, -73.91065 40.56393, -73.91069 40.56384, -73.9107 40.56342, -73.91027 40.5624, -73.90958..."
8957,9021,Queens,New York City,23003,"POLYGON ((-73.86287 40.56684, -73.86268 40.56653, -73.86268 40.56652, -73.86269 40.56651, -73.86269 40.56651, -73.86271 40.56651, -73.86272 40.56651, -73.86277 40.56658, -73.86296 40.56682, -73.86..."
8958,9022,Queens,New York City,23004,"POLYGON ((-73.85274 40.57061, -73.85312 40.57051, -73.85332 40.57044, -73.85358 40.57035, -73.85381 40.57034, -73.85385 40.57034, -73.85397 40.57032, -73.8541 40.57026, -73.85423 40.57015, -73.854..."
8959,9023,Queens,New York City,23005,"POLYGON ((-73.84878 40.57206, -73.84887 40.57202, -73.84897 40.57199, -73.84902 40.57196, -73.84907 40.57195, -73.84914 40.57192, -73.84925 40.57186, -73.84936 40.57183, -73.84941 40.5718, -73.849..."
...,...,...,...,...,...
10067,10131,Queens,New York City,40046,"POLYGON ((-73.80089 40.7692, -73.80108 40.76782, -73.80202 40.7679, -73.80295 40.76797, -73.80388 40.76804, -73.80482 40.76812, -73.80575 40.76819, -73.80669 40.76826, -73.80714 40.7683, -73.80762..."
10068,10132,Queens,New York City,40047,"POLYGON ((-73.79772 40.77856, -73.79782 40.7779, -73.79791 40.77727, -73.798 40.77659, -73.79809 40.77589, -73.79817 40.77517, -73.79821 40.77492, -73.79823 40.77486, -73.79853 40.7752, -73.79865 ..."
10069,10133,Queens,New York City,40048,"POLYGON ((-73.82467 40.77452, -73.82625 40.77453, -73.82583 40.77592, -73.82487 40.77655, -73.82433 40.77675, -73.82455 40.77527, -73.82467 40.77452))"
10070,10134,Queens,New York City,40049,"POLYGON ((-73.83695 40.76284, -73.83692 40.76277, -73.8369 40.76271, -73.8369 40.76261, -73.83679 40.76117, -73.83862 40.7571, -73.83862 40.75709, -73.83862 40.75705, -73.83867 40.75703, -73.83875..."


In [223]:
new_gdf = ed_df.merge(gdf[['county', 'geometry', 'election_district']], 
                      left_on='ed', 
                      right_on= 'election_district', 
                      how='left',
                      suffixes=('_result', '_map')
                      )


new_gdf.rename(columns={'harris': 'harris_total', 
                        'trump':'trump_total',
                        'election_district_map': 'election_district'}, inplace=True
                        )



new_gdf = new_gdf[['assembly_district', 'election_district', 'county',
                     'harris_total', 'harris_share', 
                     'trump_total', 'trump_share',
                     'geometry']]

new_gdf = new_gdf[(new_gdf.harris_total > 0) & (new_gdf.trump_total > 0)]

new_gdf = gpd.GeoDataFrame(new_gdf)

new_gdf.head()

,assembly_district,election_district,county,harris_total,harris_share,trump_total,trump_share,geometry
0,23,23001,Queens,269,0.215545,967,0.774840,"POLYGON ((-73.92033 40.56223, -73.92078 40.56148, -73.91989 40.56168, -73.91986 40.56148, -73.91982 40.56127, -73.91993 40.56126, -73.92004 40.56125, -73.92014 40.56123, -73.92013 40.561, -73.9201..."
1,23,23002,Queens,259,0.220051,907,0.770603,"POLYGON ((-73.91018 40.56493, -73.91039 40.56452, -73.91037 40.56441, -73.91055 40.56413, -73.9106 40.56403, -73.91065 40.56393, -73.91069 40.56384, -73.9107 40.56342, -73.91027 40.5624, -73.90958..."
2,23,23003,Queens,90,0.218978,318,0.773723,"POLYGON ((-73.86287 40.56684, -73.86268 40.56653, -73.86268 40.56652, -73.86269 40.56651, -73.86269 40.56651, -73.86271 40.56651, -73.86272 40.56651, -73.86277 40.56658, -73.86296 40.56682, -73.86..."
3,23,23004,Queens,352,0.295550,826,0.693535,"POLYGON ((-73.85274 40.57061, -73.85312 40.57051, -73.85332 40.57044, -73.85358 40.57035, -73.85381 40.57034, -73.85385 40.57034, -73.85397 40.57032, -73.8541 40.57026, -73.85423 40.57015, -73.854..."
4,23,23005,Queens,310,0.256835,887,0.734880,"POLYGON ((-73.84878 40.57206, -73.84887 40.57202, -73.84897 40.57199, -73.84902 40.57196, -73.84907 40.57195, -73.84914 40.57192, -73.84925 40.57186, -73.84936 40.57183, -73.84941 40.5718, -73.849..."


In [262]:
file_name = '2024_unofficial_geojson.geojson'

new_gdf.to_json("/Users/keenansmacbookairm3/Documents/GitHub/nyc_2024/"+file_name)

ValueError: Unknown na method /Users/keenansmacbookairm3/Documents/GitHub/nyc_2024/2024_unofficial_geojson.geojson

In [264]:
new_gdf.to_file("/Users/keenansmacbookairm3/Documents/GitHub/nyc_2024/semi_processed_data/"+file_name, driver="GEOJSON")